In [ ]:
import dataclasses as dc

In [ ]:
import pendulum as pdt
import parsy

In [ ]:
auto_pick_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.999.ifrac"
"""

In [ ]:
auto_pick_output_text

In [ ]:
# Utility parsers
comma = parsy.string(',') << parsy.whitespace.many()
equals = parsy.string('=')
integer = parsy.regex(r'\d+').map(int)
newline = parsy.string('\n')
left_brace = parsy.string('{')
left_paren = parsy.string('(')
right_brace = parsy.string('}')
right_paren = parsy.string(')')
single_quote = parsy.string("'")

In [ ]:
comma.parse(",")

In [ ]:
comma.parse(", ")

In [ ]:
# Single-line parsers
project_name = parsy.string("INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'")
observation_set_items = parsy.string("INFO:root:len(native_project.ObservationSets.Items)=2")
parent_well_observations = parsy.string("INFO:root:observation_set.Name='ParentWellObservations'")
get_observations = parsy.string("INFO:root:len(observation_set.GetObservations())=") >> parsy.regex(r'\d+').map(int)
auto_picked_observation_set = parsy.string("INFO:root:observation_set.Name='Auto-picked Observation Set3'")
output_path_name = (parsy.string('INFO:root:Wrote changes to') >>
                    parsy.regex(r' "c:\\src\\Orchid.IntegrationTestData\\frankNstein_Bakken_UTM13_FEET.\d{3}.ifrac"'))
unique_attribute_count_per_stage_per_well_equals = parsy.string(r'INFO:root:Unique counts of attributes per stage per well=')

In [ ]:
@parsy.generate
def attribute_count_per_stage_per_well():
    yield unique_attribute_count_per_stage_per_well_equals
    yield left_brace
    attribute_per_stage_per_well_count = yield integer
    yield right_brace
    
    return attribute_per_stage_per_well_count

In [ ]:
@parsy.generate
def get_second_observations_count():
    yield project_name
    yield newline >> observation_set_items
    yield newline >> parent_well_observations
    yield newline >> get_observations
    yield newline >> auto_picked_observation_set
    get_observations_count = yield newline >> get_observations
    yield (newline >> attribute_count_per_stage_per_well).optional()
    yield newline >> output_path_name
    yield newline

    return get_observations_count

In [ ]:
get_second_observations_count.parse(auto_pick_output_text)

In [ ]:
auto_pick_and_create_attributes_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Unique counts of attributes per stage per well={2}
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.998.ifrac"
"""

In [ ]:
@parsy.generate
def get_attribute_count_for_each_stage_and_well():
    yield project_name
    yield newline >> observation_set_items
    yield newline >> parent_well_observations
    yield newline >> get_observations
    yield newline >> auto_picked_observation_set
    yield newline >> get_observations
    attribute_count_for_each_stage_and_well = yield newline >> attribute_count_per_stage_per_well
    yield newline >> output_path_name
    yield newline

    return attribute_count_for_each_stage_and_well

In [ ]:
get_attribute_count_for_each_stage_and_well.parse(auto_pick_and_create_attributes_output_text)

In [ ]:
add_stages_output_text = r"""INFO:root:CreatedStageDetails(name='Stage-36', shmin='8144.498 psi', cluster_count=0, global_stage_sequence_no=0, start_time='2018-06-06T05:34:03.684000+00:00', stop_time='2018-06-06T07:19:35.560000+00:00')
INFO:root:CreatedStageDetails(name='Stage-37', shmin='2.322 psi', cluster_count=0, global_stage_sequence_no=0, start_time='2018-06-15T14:11:40.450000+00:00', stop_time='2018-06-15T15:10:11.200000+00:00')
INFO:root:CreatedStageDetails(name='Stage-38', shmin='8041.893 psi', cluster_count=7, global_stage_sequence_no=0, start_time='2018-06-28T23:35:54.379000+00:00', stop_time='2018-06-29T01:18:05.840000+00:00')
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.996.ifrac"
"""

In [ ]:
@parsy.generate
def key_value_pair():
    yield parsy.regex(r'[\w_\d]+').desc('key')
    value = yield equals >> ((single_quote >> parsy.regex(r"[^']+") << single_quote) |
                             integer)
    return value

In [ ]:
@dc.dataclass
class AddedStageDetails:
    stage_name: str
    shmin: str
    cluster_count: int
    global_stage_sequence_no: int
    start_time: str
    stop_time: str

@parsy.generate
def added_stage_details():
    yield parsy.string('INFO:root:CreatedStageDetails')
    yield left_paren
    stage_name = yield key_value_pair
    shmin = yield comma >> key_value_pair
    cluster_count = yield comma >> key_value_pair
    global_stage_sequence_no = yield comma >> key_value_pair
    start_time = yield (comma >> key_value_pair).map(pdt.parse).desc('start_time')
    stop_time = yield (comma >> key_value_pair).map(pdt.parse).desc('stop_time')
    yield right_paren

    return AddedStageDetails(stage_name=stage_name, shmin=shmin, cluster_count=cluster_count,
                             global_stage_sequence_no=global_stage_sequence_no,
                             start_time=start_time, stop_time=stop_time)

In [ ]:
key_value_pair.parse("name='Stage-36'")

In [ ]:
key_value_pair.parse('cluster_count=0')

In [ ]:
key_value_pair.parse("start_time='2018-06-06T05:34:03.684000+00:00'")

In [ ]:
add_stages_output_text.split('\n')[0]

In [ ]:
added_stage_details.parse(add_stages_output_text.split('\n')[0])

In [ ]:
@parsy.generate
def get_added_stages():
    added_stages_details = yield (added_stage_details << newline).at_least(1)
    yield output_path_name
    yield newline

    return added_stages_details

In [ ]:
get_added_stages.parse(add_stages_output_text)